# Detector de inperfeccións en madeira
### Iria Vázquez Álvarez e Miguel Estévez Díaz 
3º Robotica, Visión artificial

In [ ]:
import re
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix

# 1. Introduccion:

La detección de imperfecciones en la madera es un problema crucial en la industria maderera, ya que permite identificar defectos que afectan la calidad del producto final.
Este trabajo se centra en el análisis del desempeño de diferentes técnicas de detección, utilizando descriptores HOG (Histogram of Oriented Gradients) en combinación con dos clasificadores diseñados específicamente para esta tarea: un detector HOG multiescala basado en un clasificador SVM y un clasificador MLP (Perceptrón Multicapa), y un tercer clasificador que trabaja directamente sobre las imágenes, un clasificador Cascade. Todos estos clasificadores han sido implementados y ajustados manualmente como parte del desarrollo de este proyecto.

El trabajo incluye la extracción de características HOG a partir de imágenes de madera, el entrenamiento de cada uno de los clasificadores y la evaluación de su rendimiento sobre varios conjuntos de pruebas. Finalmente, estas técnicas se aplican para la detección de imperfecciones en secuencias de vídeo, lo que representa un desafío adicional debido a factores como las variaciones de iluminación, movimiento de la cámara y complejidad del fondo.

El objetivo principal es comparar la efectividad de los clasificadores mencionados en términos de precisión, recall, F1-Score y capacidad de generalización en el contexto de detección en tiempo real. Este enfoque permite identificar las fortalezas y limitaciones de cada técnica, proporcionando una base sólida para futuras mejoras e implementaciones en aplicaciones prácticas.


# 2. Apertura das imaxes:

In [ ]:
imaxes = []
clases = []
ben_cargadas = True

for i in range(1, 1252):
    ruta_pos = "Base_datos/Imaxes/Positivos/Img_" + str(i) + ".png"
    ruta_neg = "Base_datos/Imaxes/Negativos/Img_" + str(i) + ".png"

    # Procesar imagen positiva
    img = cv2.imread(ruta_pos, cv2.IMREAD_COLOR)
    if img is None:
        print(f"No se pudo cargar la imagen {ruta_pos}")
        ben_cargadas = False
        continue
    
    imaxes.append(img)
    clases.append(1)
    
    # Procesar imagen negativa
    img = cv2.imread(ruta_neg, cv2.IMREAD_COLOR)
    if img is None:
        print(f"No se pudo cargar la imagen {ruta_neg}")
        ben_cargadas = False
        continue
    
    imaxes.append(img)
    clases.append(0)
         
if ben_cargadas:
    print("Imaxes cargadas correctaente")
else:
    print("Algunha imaxe cargouse erroneamente")

A carga das imaxes realizase dende dúas carpetas diferentes, positivos e negativos, de maneira intercalada na mesma lista, para posteriormente dividilas en datos de adestramento e test

## 2.1. Division das imaxes:

In [ ]:
# División dos datos para adestramento e test
cantidad_adestramento = int(0.75 * (len(clases) / 2))
cantidad_test = 4 
cantidad_por_test = int(((len(clases) / 2) - cantidad_adestramento) / cantidad_test)

cnts = [0, 0]
cnts_2 = [[0, 0] for _ in range(cantidad_test)]

imaxes_adestramento = []
clases_adestramento = []
tests_xerados = 0
imaxes_test = [[] for _ in range(cantidad_test)]
clases_test = [[] for _ in range(cantidad_test)]

for img, clase in zip(imaxes, clases): 
    if ((cnts[0] < cantidad_adestramento and clase == 0) or (cnts[1] < cantidad_adestramento and clase == 1)):
        imaxes_adestramento.append(img)
        clases_adestramento.append(clase)
        cnts[clase] += 1     
        
        continue
    
    if tests_xerados < cantidad_test:
        if (cnts_2[tests_xerados][0] < cantidad_por_test  and cnts_2[tests_xerados][1] < cantidad_por_test):
            imaxes_test[tests_xerados].append(img)
            clases_test[tests_xerados].append(clase)
            cnts_2[tests_xerados][clase] += 1
        
            if (cnts_2[tests_xerados][0] == cantidad_por_test - 1 and cnts_2[tests_xerados][1] == cantidad_por_test - 1):
                tests_xerados += 1
           
            continue
        
print("División dos datos xerada correctamente")

La división de los datos se realiza en función de los dos primeros parámetros definidos en el bloque de código anterior, definiendo el porcentage de imagenes que queremos usar para el entrenamiento y la cantidad de tests que queremos generar con los datos restantes. En nuestro caso entrenamos el clasificador con el 75% de imagenes, es decir 1878 imaxes, dividindo as 626 imaxes restantes en 4 test de 154 imaxes cada un.

# 3. Definicion do descriptor

In [ ]:
# Parámetros para el descriptor HoG
winSize = (64, 64)
blockSize = (8, 8)
blockStride = (4, 4)
cellSize = (8, 8)
nbins = 9
derivAperture = 1
winSigma = -1.0
histogramNormType = 0
L2HysThreshold = 0.2
gammaCorrection = True
nlevels = 64

# Crear el descriptor HoG
hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins, derivAperture, winSigma, histogramNormType, L2HysThreshold, gammaCorrection, nlevels)

# Función para xerar os descriptores de Hog de una imaxe
def generar_descriptor(img):
    """Genera los descriptores HoG para un conjunto de imágenes."""
    if img is not None:
        # Redimensionar la imagen al tamaño requerido por el descriptor
        imagen_redimensionada = cv2.resize(img, winSize)

        # Convertir a escala de grises si no lo está
        if len(imagen_redimensionada.shape) == 3:
            imagen_redimensionada = cv2.cvtColor(imagen_redimensionada, cv2.COLOR_BGR2GRAY)

        # Aplicar un filtro bilateral para reducir el ruido preservando los bordes
        imagen_suavizada = cv2.bilateralFilter(imagen_redimensionada, d=9, sigmaColor=75, sigmaSpace=75)

        # Normalización de la imagen para ajustar los valores de píxeles
        imagen_normalizada = cv2.normalize(imagen_suavizada, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
            
        # Calcular el descriptor HoG
        descriptor = hog.compute(imagen_normalizada)
        
        return np.array(descriptor).flatten()
    else:
        return None 
    
print("Descriptor xerado correctamente")

HoG es una técnica utilizada para describir las características locales de una imagen mediante la distribución de los gradientes de intensidad, que incluyen tanto las direcciones como las magnitudes, en regiones específicas de la imagen. El objetivo principal de HoG es capturar las formas y bordes presentes, que son fundamentales para la identificación de objetos.

En este caso, utilizamos HoG como descriptor para transformar las imágenes de entrada (ya sean de tablas con o sin defectos) en vectores de características. Estos vectores se utilizan luego como entradas para los clasificadores (SVM y MLP). 

## Proceso para calcular el descriptor HoG

1. **Preprocesamiento de la imagen:**
   - Convertir la imagen a escala de grises (si no está ya en esa escala), ya que la intensidad de los píxeles es el dato que necesitamos para calcular los gradientes.
   - Reducir el ruido en la imagen mediante un filtro bilateral para no perder los detalles importantes.
   - Normalizar los valores de la imagen, potenciando máximos y discriminando mínimos.

2. **Cálculo de gradientes:**
   - Calcular el gradiente de la intensidad en cada píxel de la imagen utilizando operadores Sobel. Esto genera dos componentes para cada píxel:
      - **Magnitud del gradiente:** \( \sqrt{G_x^2 + G_y^2} \)
      - **Orientación del gradiente:** \( \arctan{\left(\frac{G_y}{G_x}\right)} \)


## Parámetros del Descriptor HoG

- **`winSize (64, 64):`**  
  Define el tamaño de la ventana de detección donde se calculan los descriptores HoG. La imagen se divide en varias ventanas de este tamaño, asegurando la captura de características relevantes sin perder información clave sobre bordes y formas.

- **`blockSize (8, 8):`**  
  Establece el tamaño de los bloques, que son agrupaciones de varias celdas. Cada bloque es la unidad básica normalizada en el descriptor HoG. Bloques pequeños, como 8×8, permiten una mejor resolución para capturar detalles locales.

- **`blockStride (4, 4):`**  
  Representa el desplazamiento entre bloques consecutivos. Un desplazamiento menor que el tamaño del bloque, como en este caso, introduce solapamiento entre ellos, lo que mejora la robustez frente a variaciones en la imagen.

- **`cellSize (8, 8):`**  
  Indica el tamaño de las celdas, que son las unidades donde se calculan los histogramas de los gradientes. Celdas pequeñas, como 8×8, capturan detalles más finos, pero aumentan el costo computacional.

- **`nbins (9):`**  
  Determina el número de bins en los histogramas de las orientaciones de los gradientes. Con 9 bins, el rango de 0° a 180° se divide en 9 intervalos de 20°, permitiendo capturar con precisión las direcciones de los bordes.


## Observaciones de los Resultados

A continuación se muestran parejas de imágenes y su descriptor HoG correspondiente:

<table border="1">
  <tr>
    <td><img src="Apoio/Imaxes/1_Hog.png" alt="Imagen 1" style="width:100%; height:auto;"/></td>
  </tr>
  <tr>
    <td><img src="Apoio/Imaxes/2_Hog.png" alt="Imagen 1" style="width:100%; height:auto;"/></td>
  </tr>
  <tr>
    <td><img src="Apoio/Imaxes/3_Hog.png" alt="Imagen 1" style="width:100%; height:auto;"/></td>
  </tr>
  <tr>
    <td><img src="Apoio/Imaxes/4_Hog.png" alt="Imagen 1" style="width:100%; height:auto;"/></td>
  </tr>
</table>

En ellas se observa que el descriptor HoG de la madera presenta histogramas más o menos estables, mientras que en las regiones con imperfecciones, los histogramas muestran irregularidades que representan la forma de dichas imperfecciones.

## 3.1. Creacion dos descriptores:

In [ ]:
# Descriptores adestramento
descriptores_adestramento = []
for img in imaxes_adestramento:
    descriptores_adestramento.append(generar_descriptor(img))
print("Descriptores de adestramento xerado correctamente")

# Descriptores test
descriptores_test = [[] for _ in range(cantidad_test)]
for i in range(cantidad_test):
    for img in imaxes_test[i]:
        descriptores_test[i].append(generar_descriptor(img))
        
print("Descriptores de test xerado correctamente")

La creación de los descriptores se realiza mediante la función definida en el apartado anterior, rellenando la correspondiente lista (entrenamiento o test_número) correspondiente para cada caso, manteniendo la configuración definida (números de images para cadda función y cantidades de test) inicialmente para las imagenes.

# 4. Definicion adestramento test clasificadores

In [ ]:
def mineria_negativos_dificiles(clasificador, descriptores_adestramento, clases_adestramento, tipo_clasificador):
    # Realizar prediccións sobre o conxunto de adestramento
    if tipo_clasificador == "svm":
        _, prediccions = clasificador.predict(np.array(descriptores_adestramento, dtype=np.float32))
    elif tipo_clasificador == "mlp":
        prediccions = clasificador.predict(descriptores_adestramento)
        prediccions = np.argmax(prediccions, axis=1)
    else:
        print("Tipo de clasificador no reconocido: debe ser 'svm' o 'mlp'.")
    
    # Crear datos adestramento cos falsos positivos (predicción = 1, clase = 0) e cos verdadeiros positivos
    datos_readestramento = []
    clases_readestramento = []
    cnt_falsos_positivos = 0
    for i, clase in enumerate(clases_adestramento):
        if ((prediccions[i] == 1 and clase == 0) or clase == 1):
            datos_readestramento.append(descriptores_adestramento[i])
            clases_readestramento.append(clase)
            
            if prediccions[i] == 1 and clase == 0:
                cnt_falsos_positivos += 1
            
    # Se hay falsos positivos, proceder coa minería de negativos difíciles
    if cnt_falsos_positivos > 0:
        # Readestrar o clasificador cos novos datos
        if tipo_clasificador == "svm":
            clasificador.train(np.array(datos_readestramento, dtype=np.float32), cv2.ml.ROW_SAMPLE, np.array(clases_readestramento, dtype=np.int32).reshape(-1, 1))
        elif tipo_clasificador == "mlp":
            clasificador.train(np.array(datos_readestramento, dtype=np.float32), np.array(clases_readestramento, dtype=np.int32).reshape(-1, 1))
        
        print("Readestraento completado.")
    else:
        print("Non é necesario readestrar.")



La minería de negativos difíciles consiste en identificar los falsos positivos en las predicciones de los datos de entrenamiento, después de haber entrenado el clasificador, para volver a entrenarlo con estos datos, denominados "negativos difíciles", junto con los datos positivos del entrenamiento.

El objetivo principal de esta técnica es mejorar la precisión del modelo en las predicciones finales. Al centrarse en los casos donde el clasificador está fallando, se ajusta mejor a las características críticas que diferencian los datos positivos de los negativos, haciendo que las predicciones sean más precisas.

## 4.1. Suport vector machine (SVM)

In [ ]:
reportes = []
correctas = []

# Crear e configurar o clasificador SVM con Kerner lineal
svm = cv2.ml.SVM_create()
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setC(0.1)

# Adestramento do clasificador
svm.train(np.array(descriptores_adestramento, dtype=np.float32), cv2.ml.ROW_SAMPLE, np.array(clases_adestramento).reshape(-1, 1))
print("Adestramento completado.")

# Realizar minería de negativos difíciles (Non necesaria neste caso posto que causa sobreaprencdizaxe)
#mineria_negativos_dificiles(svm, descriptores_adestramento, clases_adestramento, "svm")

# Realizar as prediccións
for i in range(cantidad_test):
    print("\nMétricas Test", i + 1)
    
    # Obter as prediccións
    _, prediccions = svm.predict(np.array(descriptores_test[i], dtype=np.float32))

    # Contar o número de imaxes correctamente clasificadas
    cnt = 0
    for j, clase in enumerate(clases_test[i]):    
        if clase == prediccions[j]:
            cnt += 1
            
    correctas.append(cnt)
            
    # Obtemos o reporte de clasificación     
    reporte = classification_report(clases_test[i], prediccions)
    reportes.append(reporte)

    # Información das métricas
    print("SVM predeciu correctamente ", cnt, " de ", len(clases_test[i]))
    print("Reporte de clasificación:", )
    print(reporte)
    print("Matriz de confusión: ")
    print(confusion_matrix(clases_test[i], prediccions))

Support Vector Machine (SVM) es un clasificador supervisado que tiene como objetivo separar los datos en diferentes clases utilizando un hiperplano óptimo en un espacio de características multidimensionales. La distancia de este hiperplano a los puntos más cercanos de cada clase (conocidos como vectores de soporte), es el margen que SVM trata de maximizar. Al maximizar este margen, el modelo logra una mejor capacidad para generalizar, es decir, para clasificar correctamente nuevos datos no vistos. En el caso de que los datos no sean separables linealmente, los SVM utilizan kernels, que son funciones de transformación, para proyectar los datos a un espacio de mayor dimensión donde sí pueden separarse. El kernel define cómo se transforman los datos para proyectarlos a un espacio de características donde sea posible separarlos, en nuestro caso usamos un kernel lineal para un problema de clasificación binaria (positivos y negativos).

En este clasificador también aplicamos minería de negativos difíciles, la cuál consiste en identificar los falsos positivos en las predicciones de los datos de entrenamiento, después de haber entrenado el clasificador, para volver a entrenarlo con estos datos, denominados "negativos difíciles", junto con los datos positivos del entrenamiento. El objetivo principal de esta técnica es mejorar la precisión del modelo en las predicciones finales. Al centrarse en los casos donde el clasificador está fallando, se ajusta mejor a las características críticas que diferencian los datos positivos de los negativos, haciendo que las predicciones sean más precisas.

El parámetro C (0.01) o parámetro de penalización controla el equilibrio entre maximizar el margen y minimizar los errores de clasificación. Valores bajos de C permiten márgenes más amplios pero con más errores, mientras que valores altos reducen los errores a costa de un margen más estrecho, lo que podría llevar al sobreajuste.

### Test 1:

| **precision**    | **recall**    | **f1-score**  | **support**  |
|------------------|---------------|---------------|--------------|
| 0                | 0.88          | 0.88          | 77           |
| 1                | 0.88          | 0.88          | 77           |
| **accuracy**     |               |               | **0.88**     |
| **macro avg**    | 0.88          | 0.88          | 154          |
| **weighted avg** | 0.88          | 0.88          | 154          |
| **Matriz de confusión** | 68 | 9 | |
|                  | 9             | 68            |              |
| SVM predijo correctamente 136 de 154 | | | |

### Test 2:

| **precision**    | **recall**    | **f1-score**  | **support**  |
|------------------|---------------|---------------|--------------|
| 0                | 0.90          | 0.95          | 77           |
| 1                | 0.95          | 0.90          | 77           |
| **accuracy**     |               |               | **0.92**     |
| **macro avg**    | 0.92          | 0.92          | 154          |
| **weighted avg** | 0.92          | 0.92          | 154          |
| **Matriz de confusión** | 73 | 4 | |
|                  | 8             | 69            |              |
| SVM predijo correctamente 142 de 154 | | | |

### Test 3:

| **precision**    | **recall**    | **f1-score**  | **support**  |
|------------------|---------------|---------------|--------------|
| 0                | 0.96          | 0.95          | 77           |
| 1                | 0.95          | 0.96          | 77           |
| **accuracy**     |               |               | **0.95**     |
| **macro avg**    | 0.95          | 0.95          | 154          |
| **weighted avg** | 0.95          | 0.95          | 154          |
| **Matriz de confusión** | 73 | 4 | |
|                  | 3             | 74            |              |
| SVM predijo correctamente 147 de 154 | | | |

### Test 4:

| **precision**    | **recall**    | **f1-score**  | **support**  |
|------------------|---------------|---------------|--------------|
| 0                | 0.94          | 0.86          | 77           |
| 1                | 0.87          | 0.95          | 77           |
| **accuracy**     |               |               | **0.90**     |
| **macro avg**    | 0.91          | 0.90          | 154          |
| **weighted avg** | 0.91          | 0.90          | 154          |
| **Matriz de confusión** | 66 | 11 | |
|                  | 4             | 73            |              |
| SVM predijo correctamente 139 de 154 | | | |

# Agregar o detector svm a HOG, para usar o método multiescale:

In [ ]:
# Obtemos o vector de soporte, o sesgo e outros parámetros do clasificador SVM
vector_soporte = svm.getSupportVectors()
rho,_,_= svm.getDecisionFunction(0)

# Creamos o detector SVM para o descritor HoG
detector_svm = np.zeros(vector_soporte.shape[1]+1, dtype=vector_soporte.dtype)
detector_svm[:-1] = -vector_soporte[:]
detector_svm[-1] = rho

# Inicializamos o descritor HoG co detector SVM
hog.setSVMDetector(detector_svm)

Finalmente conectamos el modelo SVM entrenado con el descriptor HoG para que pueda identificar defectos en las imágenes. Primero, tomamos lo que el SVM aprendió al entrenarse, es decir, qué características son importantes para diferenciar entre defectos y no defectos. Después, adaptamos esa información para que HoG pueda usarla como una especie de "regla" que le permita detectar esas mismas características en nuevas imágenes.

## 4.2. Clasificador en cascada

In [ ]:
# Cargar o clasificador en cascada
cascade = cv2.CascadeClassifier('Base_datos/cascade.xml')

# Realizar as prediccións
for i in range(cantidad_test):
    print("\nMétricas Test", i + 1)

    # Obter as prediccións
    prediccions = []
    for img in imaxes_test[i]:
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        deteccions = cascade.detectMultiScale(img, scaleFactor=1.5, minNeighbors=2, minSize=(64, 64))

        # Determinar a clase detectada (detección = lista de rectangulos positivos)
        if len(deteccions) > 0:
            prediccions.append(1)
        else:
            prediccions.append(0)
    
    # Contar o número de imaxes correctamente clasificadas
    cnt = 0
    for j, clase in enumerate(clases_test[i]):
        if clase == prediccions[j]:
            cnt += 1
    
    correctas.append(cnt)
    
    # Obtemos o reporte de clasificación     
    reporte = classification_report(clases_test[i], prediccions)
    reportes.append(reporte)

    # Información das métricas
    print("Cascade predeciu correctamente ", cnt, " de ", len(clases_test[i]))
    print("Reporte de clasificación:", )
    print(reporte)
    print("Matriz de confusión: ")
    print(confusion_matrix(clases_test[i], prediccions))


El clasificador en cascada es una técnica que combina múltiples clasificadores simples en una secuencia, con el objetivo de realizar una detección eficiente. El clasificador en cascada busca la detección de los objetos mediante el entrenamiento de múltiples clasificadores débiles, basándose en este caso en descriptores HAAR.  Cada uno de los clasificadores se centra en la detección de un conjunto de características diferentes, los cuales se auto ajustan durante la fase de entrenamiento. El proceso de detección consiste en la clasificación de la imagen mediante todos los clasificadores, descartando esta en caso de alguno de ellos dar una respuesta negativa.

Generamos nuestro modelo utilizando Cascade Trainer GUI, una herramienta que simplifica la creación de clasificadores en cascada entrenados con nuestros propios datos. Esta herramienta automatiza el proceso de entrenamiento mediante el uso de imágenes positivas (que contienen el objeto de interés, las imperfecciones de la madera) e imágenes negativas (que no lo contienen). Internamente, se emplea Haar para describir las imágenes y entrenar los clasificadores. El modelo final, contenido en el archivo cascade.xml, incorpora toda la información necesaria, incluyendo umbrales, parámetros y características seleccionadas, lo que nos permite cargarlo y utilizarlo directamente con OpenCV para detectar objetos en imágenes o videos nuevos.

### Test 1:

| **precision**    | **recall**    | **f1-score**  | **support**  |
|------------------|---------------|---------------|--------------|
| 0                | 0.59          | 0.70          | 77           |
| 1                | 0.63          | 0.51          | 77           |
| **accuracy**     |               |               | **0.60**     |
| **macro avg**    | 0.61          | 0.60          | 154          |
| **weighted avg** | 0.61          | 0.60          | 154          |
| **Matriz de confusión** | 54 | 23 | |
|                  | 38            | 39            |              |
| Cascade predijo correctamente 93 de 154 | | | |


### Test 2:

| **precision**    | **recall**    | **f1-score**  | **support**  |
|------------------|---------------|---------------|--------------|
| 0                | 0.53          | 0.65          | 77           |
| 1                | 0.55          | 0.43          | 77           |
| **accuracy**     |               |               | **0.54**     |
| **macro avg**    | 0.54          | 0.54          | 154          |
| **weighted avg** | 0.54          | 0.54          | 154          |
| **Matriz de confusión** | 50 | 27 | |
|                  | 44            | 33            |              |
| Cascade predijo correctamente 83 de 154 | | | |


### Test 3:

| **precision**    | **recall**    | **f1-score**  | **support**  |
|------------------|---------------|---------------|--------------|
| 0                | 0.51          | 0.53          | 77           |
| 1                | 0.51          | 0.48          | 77           |
| **accuracy**     |               |               | **0.51**     |
| **macro avg**    | 0.51          | 0.51          | 154          |
| **weighted avg** | 0.51          | 0.51          | 154          |
| **Matriz de confusión** | 41 | 36 | |
|                  | 40            | 37            |              |
| Cascade predijo correctamente 78 de 154 | | | |


### Test 4:

| **precision**    | **recall**    | **f1-score**  | **support**  |
|------------------|---------------|---------------|--------------|
| 0                | 0.44          | 0.40          | 77           |
| 1                | 0.45          | 0.49          | 77           |
| **accuracy**     |               |               | **0.45**     |
| **macro avg**    | 0.45          | 0.45          | 154          |
| **weighted avg** | 0.45          | 0.45          | 154          |
| **Matriz de confusión** | 31 | 46 | |
|                  | 39            | 38            |              |
| Cascade predijo correctamente 69 de 154 | | | |


## 4.3. Perceptron muticapa

In [ ]:
# Crear o escalador para normalizar os datos
scaler = MinMaxScaler()

# Normalizar os descriptores de adestramento
descriptores_adestramento_normalizados = scaler.fit_transform(descriptores_adestramento)

# Crear e configurar o clasificador MLP
mlp = cv2.ml.ANN_MLP_create()
mlp.setLayerSizes(np.array([descriptores_adestramento_normalizados.shape[1], 20, 2]))  # Tamaño de las capas: 4185 entradas, 64 neuronas en capa oculta, 2 clases de salida
mlp.setActivationFunction(cv2.ml.ANN_MLP_SIGMOID_SYM)  # Función de activación sigmoide simétrica
mlp.setTermCriteria((cv2.TERM_CRITERIA_EPS, 10000, 1e-6))  # Criterios de término
mlp.setTrainMethod(cv2.ml.ANN_MLP_BACKPROP)  # Método de entrenamiento Backpropagation

# Convertir las clases a formato one-hot encoding
encoder = OneHotEncoder(sparse_output=False)
clases_adestramento_one_hot = encoder.fit_transform(np.array(clases_adestramento).reshape(-1, 1))

# Adestramento do clasificador MLP
mlp.train(np.array(descriptores_adestramento_normalizados, dtype=np.float32), cv2.ml.ROW_SAMPLE, np.array(clases_adestramento_one_hot, dtype=np.float32))
print("Adestramento completado.")

# Realizar as prediccións
for i in range(cantidad_test):
    print("\nMétricas Test", i + 1)
    
    # Normalizar os descriptores de test
    descriptores_test_normalizados = scaler.transform(np.array(descriptores_test[i], dtype=np.float32))

    # Obter as prediccións
    _, prediccions = mlp.predict(descriptores_test_normalizados)
    
    # Convertir as probabilidades en clases predicidas (tomando a clase co maior valor)
    prediccions = np.argmax(prediccions, axis=1)

    # Contar o número de imaxes correctamente clasificadas
    cnt = 0
    for j, clase in enumerate(clases_test[i]):    
        if clase == prediccions[j]:
            cnt += 1
    
    correctas.append(cnt)
    
    # Obtemos o reporte de clasificación     
    reporte = classification_report(clases_test[i], prediccions)
    reportes.append(reporte)

    # Información das métricas
    print("MLP predeciu correctamente ", cnt, " de ", len(clases_test[i]))
    print("Reporte de clasificación:")
    print(reporte)
    print("Matriz de confusión: ")
    print(confusion_matrix(clases_test[i], prediccions))

El Perceptrón Multicapa (MLP) es una red neuronal artificial supervisada diseñada para aprender relaciones no lineales entre las entradas y las salidas mediante capas de neuronas interconectadas. Consta de tres capas principales: una capa de entrada, que recibe las características (en este caso, los descriptores HoG); capas ocultas, encargadas de realizar transformaciones no lineales para identificar patrones complejos; y una capa de salida, que genera la predicción final. Las neuronas de cada capa están conectadas a las de la siguiente mediante pesos, los cuales se ajustan durante el entrenamiento para minimizar los errores de predicción..
Estructura de nuestra red -> en este caso nuestra red está compuesta por:
- Una capa de entrada con tamaño 4185, debido a que este es el tamaño de los descriptores usados.
- Una capa oculta con 20 neuronas, adecuada para aprender los patrones relevantes.
- Una capa de salida con 2 neuronas, una para cada clase.

### Test 1:

| **precision**    | **recall**    | **f1-score**  | **support**  |
|------------------|---------------|---------------|--------------|
| 0                | 0.79          | 0.77          | 77           |
| 1                | 0.77          | 0.79          | 77           |
| **accuracy**     |               |               | **0.78**     |
| **macro avg**    | 0.78          | 0.78          | 154          |
| **weighted avg** | 0.78          | 0.78          | 154          |
| **Matriz de confusión** | 59 | 18 | |
|                  | 16            | 61            |              |
| MLP predijo correctamente 120 de 154 | | | |


### Test 2:

| **precision**    | **recall**    | **f1-score**  | **support**  |
|------------------|---------------|---------------|--------------|
| 0                | 0.90          | 0.92          | 77           |
| 1                | 0.92          | 0.90          | 77           |
| **accuracy**     |               |               | **0.91**     |
| **macro avg**    | 0.91          | 0.91          | 154          |
| **weighted avg** | 0.91          | 0.91          | 154          |
| **Matriz de confusión** | 71 | 6  | |
|                  | 8             | 69            |              |
| Cascade predijo correctamente 83 de 154 | | | |


### Test 3:

| **precision**    | **recall**    | **f1-score**  | **support**  |
|------------------|---------------|---------------|--------------|
| 0                | 0.87          | 0.88          | 77           |
| 1                | 0.88          | 0.87          | 77           |
| **accuracy**     |               |               | **0.88**     |
| **macro avg**    | 0.88          | 0.88          | 154          |
| **weighted avg** | 0.88          | 0.88          | 154          |
| **Matriz de confusión** | 68 | 9  | |
|                  | 10            | 67            |              |
| Cascade predijo correctamente 78 de 154 | | | |


### Test 4:

| **precision**    | **recall**    | **f1-score**  | **support**  |
|------------------|---------------|---------------|--------------|
| 0                | 0.88          | 0.84          | 77           |
| 1                | 0.85          | 0.88          | 77           |
| **accuracy**     |               |               | **0.86**     |
| **macro avg**    | 0.86          | 0.86          | 154          |
| **weighted avg** | 0.86          | 0.86          | 154          |
| **Matriz de confusión** | 65 | 12 | |
|                  | 9             | 68            |              |
| Cascade predijo correctamente 69 de 154 | | | |


# 5. Comparacion funcionamento

In [ ]:
# Métricas de los reportes
'''Función para extraer métricas de cada reporte'''
def procesar_reporte(reporte_texto):
    # Buscar precisión, recall, f1-score e accuracy
    precision = float(re.search(r"macro avg\s+([\d.]+)", reporte_texto).group(1))
    recall = float(re.search(r"macro avg\s+[\d.]+\s+([\d.]+)", reporte_texto).group(1))
    f1_score = float(re.search(r"macro avg\s+[\d.]+\s+[\d.]+\s+([\d.]+)", reporte_texto).group(1))
    accuracy = float(re.search(r"accuracy\s+([\d.]+)", reporte_texto).group(1))
    return {"Precision": precision, "Recall": recall, "F1-Score": f1_score, "Accuracy": accuracy}

'''Transformar a lista de reportes'''
reportes_procesados = [procesar_reporte(r) for r in reportes]

'''Preparar datos para os plots'''
clasificadores = ["SVM", "Cascade", "MLP"]
                  
'''Crear gráficos por cada test'''
for test_num in range(cantidad_test):
    plt.figure(figsize=(10, 6))

    precision = [r["Precision"] for r in reportes_procesados[test_num::cantidad_test]]
    recall = [r["Recall"] for r in reportes_procesados[test_num::cantidad_test]]
    f1_score = [r["F1-Score"] for r in reportes_procesados[test_num::cantidad_test]]

    x = range(len(clasificadores))
    plt.bar(x, precision, width=0.2, label="Precision", align="center")
    plt.bar([i + 0.2 for i in x], recall, width=0.2, label="Recall", align="center")
    plt.bar([i + 0.4 for i in x], f1_score, width=0.2, label="F1-Score", align="center")

    plt.xticks([i + 0.2 for i in x], clasificadores)
    plt.title(f"Comparación de Métricas - Test {test_num + 1}")
    plt.xlabel("Clasificadores")
    plt.ylabel("Valor")
    plt.ylim(0, 1.0)
    plt.legend()
    plt.grid(axis="y")

    plt.tight_layout()
    plt.show()

# Cantidade predita correctamente
'''Gráfico para mostrar predicciones correctas por test y clasificador'''
plt.figure(figsize=(10, 6))

'''Datos para el gráfico'''
x = range(cantidad_test)
ancho_barra = 0.2  

'''Graficar las barras para cada clasificador'''
for idx, clasificador in enumerate(clasificadores):
    valores_correctos = correctas[idx::len(clasificadores)]
    posiciones = [i + (idx * ancho_barra) for i in x]
    plt.bar(posiciones, valores_correctos, width=ancho_barra, label=clasificador)

'''Configuración de la gráfica'''
plt.xticks([i + ancho_barra for i in x], [f"Test {i+1}" for i in x])
plt.title("Predicciones Correctas por Clasificador y Test")
plt.xlabel("Test")
plt.ylabel("Cantidad de Predicciones Correctas")
plt.ylim(0, cantidad_por_test * 2)
plt.legend(title="Clasificadores")
plt.grid(axis="y")

plt.tight_layout()
plt.show()

<table border="1">
  <tr>
    <td><img src="Apoio/Imaxes/5_Metricas.png" alt="Imagen 1" style="width:100%; height:auto;"/></td>
    <td><img src="Apoio/Imaxes/6_Metricas.png" alt="Imagen 1" style="width:100%; height:auto;"/></td>
  </tr>
  <tr>
    <td><img src="Apoio/Imaxes/7_Metricas.png" alt="Imagen 1" style="width:100%; height:auto;"/></td>
    <td><img src="Apoio/Imaxes/5_Metricas.png" alt="Imagen 1" style="width:100%; height:auto;"/></td>
  </tr>
  <tr>
    <td colspan="2">
      <img src="Apoio/Imaxes/9_Comparacion_resultados.png" alt="Imagen 1" style="width:100%; height:auto;"/>
    </td>
  </tr>
</table>

# 6. Prova sobre video

In [ ]:
# Umbral de Supresión de Non Máximos
conf_threshold = 0.95
nms_threshold = 0.05

# Parametros ventana deslizante multiescalr
ventana_inicial = (64, 64)
paso = 64
escalas = [1.0, 1.5, 2.0]

for i in range(1, 5):
    # Ruta e carga do vídeo
    ruta_video = f"Base_datos/Videos/Video_{i}.mp4"
    cap = cv2.VideoCapture(ruta_video)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    if not cap.isOpened():
        print(f"Non se puido abrir o vídeo {ruta_video}")
        continue
    
    # Crear o obxecto VideoWriter para gardar o video resultante
    out_video = cv2.VideoWriter(f"Apoio/Videos/Resultado_{i}.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width * 2, height * 2))
    
    while cap.isOpened():
        # Ler o fotograma actual
        ret, frame = cap.read()
        if not ret:
            break
        
        # Copiar os frames para a visualización e iniciar a lista para a posterior visualización
        frame_Hog = frame.copy()
        frame_mlp = frame.copy()
        frame_cascade = frame.copy()
        frame_original = frame.copy()
        frames = []

        # Procesar con HOG
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects_hog, scores = hog.detectMultiScale(gray, winStride=(32, 32), padding=(16, 16), scale=1.2, useMeanshiftGrouping=False)
        
        """Aplicar NMS e debuxar rectángulos"""
        rects, confidences = [], []
        for (x, y, w, h), score in zip(rects_hog, scores):
            rects.append([x, y, x + w, y + h])
            confidences.append(float(score))
        indices = cv2.dnn.NMSBoxes(rects, confidences, conf_threshold, nms_threshold)

        if isinstance(indices, np.ndarray) and len(indices) > 0: 
            for idx in indices.flatten():
                x1, y1, x2, y2 = rects[idx]
                cv2.rectangle(frame_Hog, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame_Hog, 'HOG Classifier', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 2, cv2.LINE_AA)

        # Procesar co clasificador en cascada
        deteccions = cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3, minSize=(32, 32))

        '''Convertir detecciones a formato(x, y, w, h, score)'''
        deteccions_con_scores = []
        for (x, y, w, h) in deteccions:
            score = 1.0  
            deteccions_con_scores.append([x, y, x + w, y + h, score])

        '''Aplicar supresión de no máximos'''
        deteccions_con_scores = np.array(deteccions_con_scores)
        indices = cv2.dnn.NMSBoxes(bboxes=deteccions_con_scores[:, :4].tolist(), scores=deteccions_con_scores[:, 4].tolist(), score_threshold=0.5, nms_threshold=0.4)

        '''Dibujar solo las detecciones retenidas'''
        if len(indices) > 0:
            for m in indices.flatten():
                x1, y1, x2, y2 = deteccions_con_scores[m :4].astype(int)
                cv2.rectangle(frame_cascade, (x1, y1), (x2, y2), (0, 255, 0), 2)

        cv2.putText(frame_cascade, 'Cascade Classifier', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 2, cv2.LINE_AA)

        # Procesar co MLP
        rects_mlp = []
        confidences_mlp = []
        for escala in escalas:
            frame_escalado = cv2.resize(frame, (int(frame.shape[1] * escala), int(frame.shape[0] * escala)))
            for y in range(0, frame_escalado.shape[0] - ventana_inicial[1], paso):
                for x in range(0, frame_escalado.shape[1] - ventana_inicial[0], paso):
                    roi = frame_escalado[y:y + ventana_inicial[1], x:x + ventana_inicial[0]]
                    descriptor = generar_descriptor(roi)
                    if descriptor is not None:
                        des_normalizados = scaler.transform(np.array(descriptor, dtype=np.float32).reshape(1, -1))
                        _, prediccion = mlp.predict(np.array(des_normalizados, dtype=np.float32))
                        probabilidade_clase = np.max(prediccion, axis=1)
                        if 0 in np.argmax(prediccion, axis=1):  # Se predice a clase obxecto
                            x1 = int(x / escala)
                            y1 = int(y / escala)
                            x2 = int((x + ventana_inicial[0]) / escala)
                            y2 = int((y + ventana_inicial[1]) / escala)
                            rects_mlp.append([x1, y1, x2, y2])
                            confidences_mlp.append(float(probabilidade_clase[0]))
        
        """Aplicar NMS para o MLP"""
        indices_mlp = cv2.dnn.NMSBoxes(rects_mlp, confidences_mlp, conf_threshold, nms_threshold)

        if isinstance(indices_mlp, np.ndarray) and len(indices_mlp) > 0:  # Verifica se hai deteccións válidas
            for idx in indices_mlp.flatten():
                x1, y1, x2, y2 = rects_mlp[idx]
                cv2.rectangle(frame_mlp, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame_mlp, 'MLP Classifier', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 2, cv2.LINE_AA)

        # Escribir "original" no frame orixinal
        cv2.putText(frame_original, 'Original', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 2, cv2.LINE_AA)

        # Engadir frames procesados ao mosaico
        frames.append([frame_original, frame_Hog, frame_cascade, frame_mlp])

        filas = 2
        columnas = 2
        altura, ancho = frames[0][0].shape[:2]
        mosaico = np.zeros((filas * altura, columnas * ancho, 3), dtype=np.uint8)
        
        # Añadir os frames no mosaico
        for j, frame_set in enumerate(frames):
            for j, frame in enumerate(frame_set):
                fila = j // columnas
                columna = j % columnas
                mosaico[fila * altura:(fila + 1) * altura, columna * ancho:(columna + 1) * ancho, :] = frame
        
        # Frame procesado + deteccións dos clasificadores
        out_video.write(mosaico)
                
    cap.release()
    out_video.release()
    
    print("Video", i, "generado correctamente")

## !EJECUTE LA SIGUIENTE CELDA PARA QUE SE REPRODUZCAN LOS VIDEOS RESULTANTES¡
## Celda de reproducción de los videos:

In [ ]:
# Parámetros para o vídeo combinado
fps_saida = 30
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Dimensións da xanela de visualización
ancho_visualizacion = 640
alto_visualizacion = 360

# Función para crear un frame gris con texto
def crear_frame_transicion(ancho, alto, texto):
    frame = np.full((alto, ancho, 3), (50, 50, 50), dtype=np.uint8)
    cv2.putText(frame, texto, (int(ancho * 0.25), int(alto * 0.5)), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3, cv2.LINE_AA)  # Texto branco
    return frame

# Bucle de reprodución e creación do vídeo
for i in range(1, 5):
    print("Reproduciendo video", i)
    
    # Ruta e carga do vídeo
    ruta_video = f"Apoio/Videos/Resultado_{i}.mp4"
    cap = cv2.VideoCapture(ruta_video)
    fps_video = cap.get(cv2.CAP_PROP_FPS)
    ancho = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    alto = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    if not cap.isOpened():
        print(f"Non se puido abrir o vídeo {ruta_video}")
        continue

    # Transición: frame gris con texto
    frame_transicion = crear_frame_transicion(ancho, alto, f"Resultados_{i}")
    for _ in range(fps_saida):
        frame_redimensionado = cv2.resize(frame_transicion, (ancho_visualizacion, alto_visualizacion))
        cv2.imshow("Resultados", frame_redimensionado)
        if cv2.waitKey(30) in [ord('q'), ord('Q'), 27]:
            break

    # Reproducir e gardar o vídeo actual
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_redimensionado = cv2.resize(frame, (ancho_visualizacion, alto_visualizacion))
        cv2.imshow("Resultados", frame_redimensionado)
        if cv2.waitKey(30) in [ord('q'), ord('Q'), 27]:
            break

    cap.release()

cv2.destroyAllWindows()
